# Music STEMS Final Project
## Kai Kaneshina and Gabriel Quiroz

In [1]:
import numpy as np
import librosa as lb
from librosa import display
import matplotlib.pyplot as plt
import IPython.display as ipd
import scipy.signal as ss
import scipy.io as sio
import glob
import subprocess
import os.path
import operator
import pickle

## Database Constructor

In [2]:
def constructDatabase(indir):
    """
    Construct a database of fingerprints for all mp3 files in the specified directory.
    
    Arguments:
    indir -- directory containing mp3 files
    
    Returns:
    d -- database of artist names, where the key is the artist name
        and the value is the audio array extracted with librosa.
    """
    d = {}

    ### START CODE BLOCK ###
    path = indir + '/*.m4a'
    for filename in glob.iglob(path, recursive=True):  
        audio, sr = lb.core.load(filename, 22050)
        filename = filename.lstrip(indir + '/') #removes the directory from the filename
        name = filename.split(' ')[0] #keeps only the name of the artist
        d[name] = audio
            
    ### END CODE BLOCK ###
    
    return d

In [3]:
db_Scales = constructDatabase('scales') # note that this may take a minute or so to run

In [4]:
with open('db_Scales.pkl','wb') as scales:
    pickle.dump(db_Scales, scales)

In [5]:
with open('db_Scales.pkl','rb') as scales:
    db_Scales = pickle.load(scales)

In [6]:
db_JingleBells = constructDatabase('song') # note that this may take a minute or so to run

In [7]:
with open('db_JingleBells.pkl','wb') as JingleBells:
    pickle.dump(db_JingleBells, JingleBells)

In [8]:
with open('db_JingleBells.pkl','rb') as JingleBells:
    db_JingleBells = pickle.load(JingleBells)

## Functions for Visualization 

In [9]:
def visualizeTemplate(W):
    '''This function allows us the visualize the template matrix (W)'''
    fs = 22050
    winsz = 2048
    
    maxFreq = W.shape[0]*fs/winsz
    maxMidi = W.shape[1]
    
    plt.figure(figsize=(16,5))
    plt.imshow(W, cmap='jet', origin = 'lower', aspect='auto', extent=(0, maxMidi, 0, maxFreq))
    plt.xlabel('Note')
    plt.ylabel('Frequency [Hz]')
    plt.title('Template Matrix Visualization')
    plt.colorbar()
    plt.show()

In [10]:
def visualizeActivations(H):
    '''This function allows us the visualize the activations matrix (H)'''
    fs = 22050
    winsz = 2048
    
    maxNote = H.shape[0]
    maxFrame = H.shape[1]
    
    plt.figure(figsize=(16,5))
    plt.imshow(H, cmap='jet', origin = 'lower', aspect='auto', extent=(0, maxFrame, 0, maxNote))
    plt.xlabel('Frames')
    plt.ylabel('Note')
    plt.title('Activation Matrix Visualization')
    plt.colorbar()
    plt.show()

In [11]:
def visualizeSTFT(STFT):
    '''This function allows us to visualize the log spectrogram of the STFT'''
    fs = 22050
    winsz = 2048
    
    maxFreq = STFT.shape[0]*fs/winsz
    maxFrame = STFT.shape[1]
    
    plt.figure(figsize=(16,5))
    plt.imshow(np.log(STFT), cmap='jet', vmin=-12, origin = 'lower', aspect='auto', extent=(0, maxFrame, 0, maxFreq))
    plt.xlabel('Frame')
    plt.ylabel('Frequency [Hz]')
    plt.title('STFT Visualization')
    plt.colorbar()
    plt.show()

## NMF

In [12]:
def calcSTFT(audio, sr = 22050, winsz = 2048, hop = 512):
    '''Calculate the STFT of the audio'''
    f, t, Zxx = ss.stft(audio, sr, nperseg=winsz, noverlap=winsz-hop)
    return Zxx

In [13]:
def NMF(W, H, V):
    '''The NMF function runs the algorithm until the NMF algorithm causes a change of less than 1e-6 '''
    
    count = 0
    while(True):
        prevEstimate = np.linalg.norm(V - np.matmul(W,H))
        
        H = (H*np.matmul(W.T, V))/(np.matmul(np.matmul(W.T, W), H))
        W = (W*np.matmul(V, H.T))/(np.matmul(np.matmul(W, H), H.T))
        
        count += 1
        newEstimate = np.linalg.norm(V - np.matmul(W,H))
        
        # Take the difference of the previous and newly computed values 
        # If the number is small, then that means there has been little to no change within the NMF algorithm
        # and thus we can stop running the function
        if np.abs(newEstimate - prevEstimate) <= 1e-6:
            break
    return W, H, count 

## Initializing templates

In [14]:
def initTemplates(STFT, midiArray, deltaF=30, sr=22050, winSize=2048):
    '''Creates the initial template for the W matrix'''
    # Convert the midi array values into a frequency array
    freqArray = 440 * pow(2, (midiArray-69)/12)
    
    # Convert the frequency array into an array of k values. Also convert deltaF into a k value as well.
    kArray = np.round(freqArray*winSize/sr).astype(int)
    deltaK = np.round(deltaF*winSize/sr).astype(int)
    
    # Create W with the same amount of rows as the STFT, and the amount of columns equal to number of notes
    W = np.zeros([STFT.shape[0], len(kArray)])
    
    # Loop through the rows in each column
    for i in range(W.shape[1]):
        for j in range(W.shape[0]):
            
            # If our k value is a multiple of the row, we have hit the note, and/or its harmonic
            if (j%kArray[i]==0):
                
                # Set the W array row equal to the max value
                W[j,i] = 1 #np.random.rand()
                
                # Cases for harmonics are below, first init an array for the k values required to satisfy values
                # around harmonic frequencies
                deltaKRange = np.arange(1, (deltaK+1)*j/kArray[i])
                
                # We add the harmonic value to a range of k values based off of the deltaF entered
                harmonicsPositive = j + deltaKRange
                
                # We then remove any k values that are greater than or equal to the amount of rows  
                harmonicsPositive = np.delete(harmonicsPositive, np.where(harmonicsPositive>=W.shape[0])).astype(int)
                
                # We subtract a range of k values based off of the deltaF entered from the harmonic value
                harmonicsNegative = j - deltaKRange
                
                # We then remove any k values that are less than 0 to avoid the out of bounds error  
                harmonicsNegative = np.delete(harmonicsNegative, np.where(harmonicsNegative<0)).astype(int)
                
                # If an array is non-empty, we set the harmonic value + the deltaK values to be a random number

                if harmonicsPositive.size > 0:
                    # Create as many random numbers as indices selected
                    W[harmonicsPositive, i] = np.ones(harmonicsPositive.size)
                    
                if harmonicsNegative.size > 0:
                    # Create as many random numbers as indices selected
                    W[harmonicsNegative, i] = np.ones(harmonicsNegative.size)
            
    return W        

## Activation Initialization

In [15]:
def naiveMatrixInit(STFT):
    '''This function fills in the H matrices with ones'''
    numRows, numCols = STFT.shape
    numNotes = 8
    H = np.ones([numNotes, numCols])
    return H

In [16]:
def initActivationMatrix(STFT, fs=22050, hopsize=512, deltaT=.5):
    '''This function allows us to initialize the H matrix by utilizing the STFT and assuming equal note duration.'''
    
    # Parameters to save for creating the H Matrix.
    numRows, numCols = STFT.shape
    numNotes = 8
    
    # Assume that the notes last an equal amount of time.
    noteDuration = int(numCols/numNotes)
    
    deltaFrames = int(deltaT*fs/hopsize) #converts the deltaT (onset value) into frames
    
    # Create the H Matrix
    H = np.zeros([numNotes, numCols])
    
    # Create a range of values that span from the -onset value to the noteDuration + the onset value
    noteLengthInFrames = np.arange(-deltaFrames, noteDuration + deltaFrames + 1)
    
    # The note initially starts at frame 0.
    actFrame = 0
    for i in range(H.shape[0]):
        
        # Creates an array of frame values that need to be set to ones
        noteTimeActive = actFrame + noteLengthInFrames
        
        # Remove frames that are less than zero and greater than shape of activation matrix
        noteTimeActive = np.delete(noteTimeActive, np.where(noteTimeActive < 0)).astype(int)
        noteTimeActive = np.delete(noteTimeActive, np.where(noteTimeActive >= H.shape[1])).astype(int)
        
        # Set the H matrix with as many ones as needed
        H[i, noteTimeActive] = np.ones(noteTimeActive.size)
        
        # Increase the actFrame by the noteDuration for setting the next note.
        actFrame += noteDuration
    return H

## Score Metric

In [17]:
def accuracyScore(queryStftMag, queryActivation, template):
    
    '''
    This function calculates and returns the score for how accurately a scale matches up with the query.
    '''
    
    vEstimate = np.matmul(template, queryActivation)
    score = np.linalg.norm(queryStftMag - vEstimate)
    
    return score

## Initiate template and activation matrices for the scales

In [18]:
def getAudioTemplatesNMF(db_Scales, midiNotes):

    '''
    This function initializes a random H matrix and a W matrix for the audio based on the scale of the midi
    notes, as well as the STFT, which are used for the NMF function. The function returns a database with the 
    artist's name as the key and W matrix as the value. 
    '''
    
    # Initialize W 
    dbW = {}
    
    # Loop through dictionary's items
    for artist, audio in db_Scales.items():

        # Calculations required for NMF 
        STFT = calcSTFT(audio)
        magSTFT = np.abs(STFT)
        initW = initTemplates(magSTFT, midiNotes)
        
        # Initialize Activation Matrix
        randH = initActivationMatrix(STFT)
        
        # Run the NMF algorithm
        W, H, count = NMF(initW + 1e-9, randH + 1e-9, magSTFT)

        # Save the H, W, and STFT magnitude values to their respective dictionaries.
        dbW[artist] = W

    return dbW

In [19]:
def calculateArtistActivation(W, V):
    '''
    The NMF function with a fixed W matrix runs the algorithm until it causes a change of less than 1e-6. Used to
    create the Artist Activation matrix when we have a fixed template matrix (W).
    '''
    
    H = naiveMatrixInit(V)
    count = 0
    while(True):
        prevEstimate = np.linalg.norm(V - np.matmul(W,H))
        
        H = (H*np.matmul(W.T, V))/(np.matmul(np.matmul(W.T, W), H))
        
        count += 1
        newEstimate = np.linalg.norm(V - np.matmul(W,H))
        
        # Take the difference of the previous and newly computed values 
        # If the number is small, then that means there has been little to no change within the NMF algorithm
        # and thus we can stop running the function
        if np.abs(newEstimate - prevEstimate) <= 1e-6:
            break
    return H

In [20]:
def calculateSinger(db_Queries, db_Templates):
    
    '''
    This function iterates through all the queries, calculates the stft of the query, and then iterates through all
    the artist templates. It then calculates the optimal activation matrix as well as the corresponding score. 
    The lowest score calculated equates to the best artist template that matches the query, and we store the query 
    artist name with the template name and its score into a dictionary. The function then returns the dictionary.
    '''
    
    # Initialize pairing dictionary
    singerPairings = {}
    
    # Loop through the query dictionary's items
    for artistQuery, audio in db_Queries.items():
        
        print('Query Artist Name: ' , artistQuery)
        
        # Calculate STFT for the query
        STFT = calcSTFT(audio)
        magSTFT = np.abs(STFT)
        bestArtist = ''
        lowestScore = np.inf
        
        # Iterate through all the templates, 
        for artistTemplate, template in db_Templates.items():
            
            artistActivation = np.matmul(np.linalg.inv(np.matmul(template.T, template)), 
                                     np.matmul(template.T, magSTFT))
            
            score = accuracyScore(magSTFT, artistActivation, template)
            
            print('Template Name and Score: ', artistTemplate, ', ', score)
            
            if score < lowestScore:
                bestArtist = artistTemplate
                lowestScore = score
                
        print('Best Template Match: ', bestArtist)        
        singerPairings[artistQuery] = (bestArtist, lowestScore)
            
    return singerPairings

In [21]:
def calculateSingerStoreAll(db_Queries, db_Templates):
    
    '''
    This function iterates through all the queries, calculates the stft of the query, and then iterates through all
    the artist templates. It then calculates the optimal activation matrix as well as the corresponding score. 
    We store the query artist name with all of the template names and their scores into a dictionary. 
    The function then returns the dictionary.
    '''
    
    # Initialize pairing dictionary
    singerPairings = {}
    
    # Loop through the query dictionary's items
    for artistQuery, audio in db_Queries.items():
        
        print('Query Artist Name: ' , artistQuery)
        
        # Calculate STFT for the query
        STFT = calcSTFT(audio)
        magSTFT = np.abs(STFT)
        artistsAndScores = []
        
        # Iterate through all the templates, 
        for artistTemplate, template in db_Templates.items():
            
            artistActivation = np.matmul(np.linalg.inv(np.matmul(template.T, template)), 
                                     np.matmul(template.T, magSTFT))
            
            score = accuracyScore(magSTFT, artistActivation, template)
            
            print('Template Name and Score: ', artistTemplate, ', ', score)
            
            artistsAndScores.append((artistTemplate, score))
                   
        singerPairings[artistQuery] = artistsAndScores
            
    return singerPairings

In [22]:
midiNotes = np.array([60, 62, 64, 65, 67, 69, 71, 72]) # This list contains the midi values for notes C4 to C5 aka 60 to 72, skipping sharps

In [23]:
db_Templates = getAudioTemplatesNMF(db_Scales, midiNotes)

In [24]:
singerPairings = calculateSinger(db_JingleBells, db_Templates)

Query Artist Name:  Isabel
Template Name and Score:  Sabrina ,  0.14801294139487473
Template Name and Score:  Cynthia ,  0.15301911578732832
Template Name and Score:  Jane ,  0.14270143801386015
Template Name and Score:  Kimi ,  0.13807210583251808
Template Name and Score:  Rachel ,  0.1521586787660821
Template Name and Score:  Maya ,  0.17459330266403428
Template Name and Score:  Lucy ,  0.1347336058708178
Template Name and Score:  AaronT ,  0.17291736423409323
Template Name and Score:  Kaitlyn ,  0.1446036274199767
Template Name and Score:  Isabel ,  0.15420826502675838
Template Name and Score:  Shiv ,  0.17141187167935953
Template Name and Score:  Maria ,  0.16623412960730222
Template Name and Score:  Mason ,  0.18256433273634853
Template Name and Score:  Gary ,  0.18073189799172737
Template Name and Score:  Djassi ,  0.17998139844023864
Template Name and Score:  Lilly ,  0.15758414714107097
Template Name and Score:  Mario ,  0.1708384668529526
Template Name and Score:  Abtin ,  0.1

Template Name and Score:  Sabrina ,  0.7302989690331301
Template Name and Score:  Cynthia ,  0.7262167703209143
Template Name and Score:  Jane ,  0.620085592299322
Template Name and Score:  Kimi ,  0.6251105550429483
Template Name and Score:  Rachel ,  0.6486033022329717
Template Name and Score:  Maya ,  0.693515911421413
Template Name and Score:  Lucy ,  0.7210891448050791
Template Name and Score:  AaronT ,  0.7619853604614387
Template Name and Score:  Kaitlyn ,  0.6574909965180264
Template Name and Score:  Isabel ,  0.6524631280145359
Template Name and Score:  Shiv ,  0.754411268134643
Template Name and Score:  Maria ,  0.768895529641505
Template Name and Score:  Mason ,  0.7188736134984662
Template Name and Score:  Gary ,  0.6607814817654019
Template Name and Score:  Djassi ,  0.7516930634049988
Template Name and Score:  Lilly ,  0.6423356640925183
Template Name and Score:  Mario ,  0.6514233817755354
Template Name and Score:  Abtin ,  0.7491421878308842
Template Name and Score:  Ga

Template Name and Score:  Abtin ,  1.809001065046182
Template Name and Score:  Gabe ,  1.7583979724994563
Template Name and Score:  Kai ,  1.8387792532153802
Template Name and Score:  Chris ,  1.9887160212192505
Template Name and Score:  Kailee ,  1.9132422907505604
Best Template Match:  Gary
Query Artist Name:  Mason
Template Name and Score:  Sabrina ,  2.0675312779768173
Template Name and Score:  Cynthia ,  2.138654384409905
Template Name and Score:  Jane ,  2.0395044482142293
Template Name and Score:  Kimi ,  1.9142350245059125
Template Name and Score:  Rachel ,  1.9884241619310612
Template Name and Score:  Maya ,  2.218697331140505
Template Name and Score:  Lucy ,  2.200556318126553
Template Name and Score:  AaronT ,  1.9947686532169069
Template Name and Score:  Kaitlyn ,  2.0450538860273824
Template Name and Score:  Isabel ,  2.183954340498626
Template Name and Score:  Shiv ,  2.0111799492033757
Template Name and Score:  Maria ,  2.1341556019947845
Template Name and Score:  Mason 

In [25]:
singerPairingsTotal = calculateSingerStoreAll(db_JingleBells, db_Templates)

Query Artist Name:  Isabel
Template Name and Score:  Sabrina ,  0.14801294139487473
Template Name and Score:  Cynthia ,  0.15301911578732832
Template Name and Score:  Jane ,  0.14270143801386015
Template Name and Score:  Kimi ,  0.13807210583251808
Template Name and Score:  Rachel ,  0.1521586787660821
Template Name and Score:  Maya ,  0.17459330266403428
Template Name and Score:  Lucy ,  0.1347336058708178
Template Name and Score:  AaronT ,  0.17291736423409323
Template Name and Score:  Kaitlyn ,  0.1446036274199767
Template Name and Score:  Isabel ,  0.15420826502675838
Template Name and Score:  Shiv ,  0.17141187167935953
Template Name and Score:  Maria ,  0.16623412960730222
Template Name and Score:  Mason ,  0.18256433273634853
Template Name and Score:  Gary ,  0.18073189799172737
Template Name and Score:  Djassi ,  0.17998139844023864
Template Name and Score:  Lilly ,  0.15758414714107097
Template Name and Score:  Mario ,  0.1708384668529526
Template Name and Score:  Abtin ,  0.1

Template Name and Score:  Abtin ,  0.14541189160334558
Template Name and Score:  Gabe ,  0.13430047419159255
Template Name and Score:  Kai ,  0.13729000009773304
Template Name and Score:  Chris ,  0.14557545797865726
Template Name and Score:  Kailee ,  0.1393999036998883
Query Artist Name:  Kaitlyn
Template Name and Score:  Sabrina ,  0.8311937919313559
Template Name and Score:  Cynthia ,  0.7297956021617175
Template Name and Score:  Jane ,  0.6202916403944843
Template Name and Score:  Kimi ,  0.726567594324961
Template Name and Score:  Rachel ,  0.6061009308212366
Template Name and Score:  Maya ,  0.9195630309298196
Template Name and Score:  Lucy ,  0.6503613119606512
Template Name and Score:  AaronT ,  0.9913338098390444
Template Name and Score:  Kaitlyn ,  0.538545513162292
Template Name and Score:  Isabel ,  0.9444877671355557
Template Name and Score:  Shiv ,  1.0010954298748198
Template Name and Score:  Maria ,  0.8944884791346295
Template Name and Score:  Mason ,  0.9856189755397

Template Name and Score:  Maria ,  0.20756081576762103
Template Name and Score:  Mason ,  0.17118784326362002
Template Name and Score:  Gary ,  0.1654096661122946
Template Name and Score:  Djassi ,  0.16924907922598337
Template Name and Score:  Lilly ,  0.1910689031965211
Template Name and Score:  Mario ,  0.17244869518097225
Template Name and Score:  Abtin ,  0.1865916444443698
Template Name and Score:  Gabe ,  0.18993663563802665
Template Name and Score:  Kai ,  0.18535625065835712
Template Name and Score:  Chris ,  0.19979747724410385
Template Name and Score:  Kailee ,  0.18461432685982146
Query Artist Name:  Maya
Template Name and Score:  Sabrina ,  0.20365049681902325
Template Name and Score:  Cynthia ,  0.19362492406326287
Template Name and Score:  Jane ,  0.20734388739138268
Template Name and Score:  Kimi ,  0.216346138955
Template Name and Score:  Rachel ,  0.20716101715666324
Template Name and Score:  Maya ,  0.21256211201607506
Template Name and Score:  Lucy ,  0.199164489489

Template Name and Score:  Mason ,  0.6783999457163067
Template Name and Score:  Gary ,  0.6773358890384452
Template Name and Score:  Djassi ,  0.6767722671889387
Template Name and Score:  Lilly ,  0.6158388726217584
Template Name and Score:  Mario ,  0.7071912641591945
Template Name and Score:  Abtin ,  0.6866497073904629
Template Name and Score:  Gabe ,  0.6750757649749078
Template Name and Score:  Kai ,  0.7138537546062768
Template Name and Score:  Chris ,  0.7209889291492065
Template Name and Score:  Kailee ,  0.6554373874041536
Query Artist Name:  Gabe
Template Name and Score:  Sabrina ,  0.45482022028653085
Template Name and Score:  Cynthia ,  0.45463500483493174
Template Name and Score:  Jane ,  0.44684676881192814
Template Name and Score:  Kimi ,  0.4298174795704503
Template Name and Score:  Rachel ,  0.4294280684108246
Template Name and Score:  Maya ,  0.482124441257347
Template Name and Score:  Lucy ,  0.4553055135349325
Template Name and Score:  AaronT ,  0.4323323555748137
T

## Accuracy Testing

In [26]:
def accuracyTest(singerPairings):
    
    '''
    This function is used to test the accuracy of the system.
    '''
    
    # Initialize variables
    amountCorrect = 0
    totalSongs = len(singerPairings)
    
    # Loop through the singerPairings database 
    for querySinger, resultsSingerAndScore in singerPairings.items():
        
        if querySinger == resultsSingerAndScore[0]:
            amountCorrect += 1
    
    return amountCorrect/totalSongs

In [27]:
def accuracyTestAll(singerPairingsTotal, xAmount):
    
    '''
    This function is used to test the accuracy of the system by seeing if the matching artist is 
    within the top xAmount scores.
    '''
    
    # Initialize variables
    amountCorrect = 0
    totalSongs = len(singerPairingsTotal)
    
    # Loop through the singerPairingsTotal database 
    for querySinger, resultsSingersAndScores in singerPairingsTotal.items():
        
        # Sort the values by the second entry of the tuple
        resultsSingersAndScores.sort(key=operator.itemgetter(1))
        
        # Loop through the resultsSingersAndScores as many times as specified by the user.
        for i in range(xAmount):
            
            singer = resultsSingersAndScores[i][0]
            
            if singer == querySinger:
                amountCorrect += 1
                print('score was in the top', xAmount, 'for: ', querySinger)
        
    return amountCorrect/totalSongs

In [28]:
accuracy = accuracyTest(singerPairings)

In [29]:
print(accuracy)

0.3181818181818182


In [30]:
accuracyTop3 = accuracyTestAll(singerPairingsTotal,3)

score was in the top 3 for:  Cynthia
score was in the top 3 for:  Kailee
score was in the top 3 for:  Mario
score was in the top 3 for:  Kaitlyn
score was in the top 3 for:  Rachel
score was in the top 3 for:  Jane
score was in the top 3 for:  Lucy
score was in the top 3 for:  Gary
score was in the top 3 for:  Mason
score was in the top 3 for:  AaronT
score was in the top 3 for:  Djassi
score was in the top 3 for:  Chris
score was in the top 3 for:  Kimi


In [31]:
print(accuracyTop3)

0.5909090909090909


In [32]:
accuracyTop4 = accuracyTestAll(singerPairingsTotal,4)

score was in the top 4 for:  Cynthia
score was in the top 4 for:  Kailee
score was in the top 4 for:  Maria
score was in the top 4 for:  Kai
score was in the top 4 for:  Mario
score was in the top 4 for:  Kaitlyn
score was in the top 4 for:  Lilly
score was in the top 4 for:  Rachel
score was in the top 4 for:  Jane
score was in the top 4 for:  Lucy
score was in the top 4 for:  Gary
score was in the top 4 for:  Mason
score was in the top 4 for:  AaronT
score was in the top 4 for:  Djassi
score was in the top 4 for:  Chris
score was in the top 4 for:  Kimi


In [33]:
print(accuracyTop4)

0.7272727272727273


In [34]:
def accuracyTestMultiple(db_JingleBells, db_Scales, midiNotes, numTests):
    '''
    This function is used to run multiple tests on the databases of scales and songs. It outputs the average 
    accuracy after numTests are run.
    '''
    
    totalAccuracy = 0
    totalIterations = numTests
    
    while(numTests>=1):
        # Get the templates based off the scales
        db_Templates = getAudioTemplatesNMF(db_Scales, midiNotes)
        
        # Create the singer pairings dictionary 
        singerPairings = calculateSinger(db_JingleBells, db_Templates)
        
        # Get the overall accuracy
        totalAccuracy += accuracyTest(singerPairings)
        
        numTests -= 1
        
    return totalAccuracy / totalIterations  

In [35]:
# accuracy10Tests = accuracyTestMultiple(db_JingleBells, db_Scales, midiNotes, 10)

In [36]:
# print(accuracy10Tests)

# Archive

## Midi Notes and Timing

In [37]:
def noteToIdx(note):
    '''
    This function will be used to transcribe the given note into an index value (0-11). 
    '''
    
    if note == 'C':
        return 0
    elif note=='C#': 
        return 1
    elif note=='D':
        return 2
    elif note=='D#':
        return 3
    elif note=='E':
        return 4
    elif note=='F':
        return 5
    elif note=='F#':
        return 6
    elif note=='G':
        return 7
    elif note=='G#':
        return 8
    elif note=='A':
        return 9
    elif note=='A#':
        return 10
    else:
        # Case for when the note is B
        return 11

In [38]:
def notesToIdx(noteArray):
    
    '''
    This function will be used to transcribe the given notes into an index value (0-11) array. 
    '''
    
    noteIdxArray = []
    
    for i in range(len(noteArray)):
        
        noteIdxArray.append(noteToIdx(noteArray[i]))
    
    return np.array(noteIdxArray)

In [39]:
def createNoteEvents(notes,noteStartTimesArray):
    
    '''
    This function is used to convert the notes into indices and then 
    combine the note indices with their corresponding start times.
    '''
    
    noteEvents = []
    
    noteIdxArray = notesToIdx(notes)

    for i in range(len(noteIdxArray)):
        
        noteEvents.append((noteIdxArray[i], noteStartTimesArray[i]))
    
    return noteEvents

In [40]:
# Notes and timing for Jingle Bells were transcribing using the chorus found here: 
# https://www.bethsnotesplus.com/2014/07/jingle-bells.html

notes = np.array(['E', 'E', 'E', 'E', 'E', 'E', 'E', 'G', 'C', 'D', 'E', 'F', 'F', 'F', 'F', 'F', 'E', 'E',   
                  'E', 'E', 'E', 'D', 'D', 'E', 'D'])
    
    #, 'G'
    
notesB = np.array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'D', 'G', 'A', 'B', 'C', 'C', 'C', 'C', 'C', 'B', 
                   'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'D','B', 'B', 'B', 'B', 'B', 'B', 'B', 'D', 
                   'G', 'A', 'B', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'B', 'B', 'D', 'D', 'C', 'A', 'G'])

noteTiming = np.array([0,1,2,4,5,6,8,9,10,11.5,12,17,18,19.5,20,21,22,23,23.5,24,25,26,27,28,30,32,33,34,36,
                       37,38,40,41,42,43.5,44,48,49,50,51.5,52,53,54,55,55.5,56,57,58,59,60,64])

noteStartTimes = np.array([0, 0.24,  0.48,  0.96,  1.2 ,  1.44,  1.92,  2.16,  2.4 ,  2.76,
        2.88,  4.08,  4.32,  4.68,  4.8 ,  5.04,  5.28,  5.52,  5.64,
        5.76,  6.  ,  6.24,  6.48,  6.72,  7.2 ,  7.68,  7.92,  8.16,
        8.64,  8.88,  9.12,  9.6 ,  9.84, 10.08, 10.44, 10.56, 11.52,
       11.76, 12.  , 12.36, 12.48, 12.72, 12.96, 13.2 , 13.32, 13.44,
       13.68, 13.92, 14.16, 14.4 , 15.36])

In [41]:
noteEvents = createNoteEvents(notes, noteStartTimes[:len(notes)])

## Initialize Activations

In [42]:
def initActivations(Zxx, midiNotes, noteEvents, fs=22050, hopsize=512, deltaT=.5):
    '''This function allows us to initialize the H matrix by utilizing the STFT and the noteEvents.'''
    
    numNotes = len(midiNotes)
    numFrames = Zxx.shape[1]
    H = np.zeros([numNotes, numFrames])
    maxNoteLengthInFrames = int(1*fs/hopsize) #this value is for the 1 second that each note plays for
    deltaFrames = int(deltaT*fs/hopsize) #converts the deltaT (onset value) into frames
    
    # Create a range of values that span from the -onset value to the totalDuration + the onset value
    noteLengthInFrames = np.arange(-deltaFrames, maxNoteLengthInFrames + deltaFrames + 1) 
    for i in range(len(noteEvents)):
        note = noteEvents[i][0]
        actTime = noteEvents[i][1] #time when the note is activated 
        actFrame = int(actTime*fs/hopsize) #convert the time into frames
        
        # Creates an array of frame values that need to be set to random, non-zero numbers
        noteTimeActive = actFrame + noteLengthInFrames 
        
        # Remove frames that are less than zero and greater than shape of activation matrix
        noteTimeActive = np.delete(noteTimeActive, np.where(noteTimeActive < 0)).astype(int)
        noteTimeActive = np.delete(noteTimeActive, np.where(noteTimeActive >= H.shape[1])).astype(int)
        
        # Create as many random numbers as indices selected
        H[note, noteTimeActive] = np.random.rand(noteTimeActive.size)
        
    return H

## Experimentation

In [43]:
def NMFFixedW(W, V):
    '''The NMF function with a fixed W matrix runs the algorithm until it causes a change of less than 1e-6 '''
    
    H = naiveMatrixInit(V)
    count = 0
    while(True):
        prevEstimate = np.linalg.norm(V - np.matmul(W,H))
        
        H = (H*np.matmul(W.T, V))/(np.matmul(np.matmul(W.T, W), H))
        
        count += 1
        newEstimate = np.linalg.norm(V - np.matmul(W,H))
        
        # Take the difference of the previous and newly computed values 
        # If the number is small, then that means there has been little to no change within the NMF algorithm
        # and thus we can stop running the function
        if np.abs(newEstimate - prevEstimate) <= 1e-6:
            break
    return H

In [44]:
def calculateSingerFixedW(db_Queries, db_Templates):
    
    '''
    This function iterates through all the queries, calculates the stft of the query, and then iterates through all
    the artist templates. It then calculates the optimal activation matrix as well as the corresponding score. 
    The lowest score calculated equates to the best artist template that matches the query, and we store the query 
    artist name with the template name and its score into a dictionary. The function then returns the dictionary.
    '''
    
    # Initialize pairing dictionary
    singerPairings = {}
    
    # Loop through the query dictionary's items
    for artistQuery, audio in db_Queries.items():
        
        print('Query Artist Name: ' , artistQuery)
        
        # Calculate STFT for the query
        STFT = calcSTFT(audio)
        magSTFT = np.abs(STFT)
        bestArtist = ''
        lowestScore = np.inf
        
        # Iterate through all the templates, 
        for artistTemplate, template in db_Templates.items():
            
            artistActivation = NMFFixedW(template, magSTFT)
            
            curScore = accuracyScore(magSTFT, artistActivation, template)
            print('Template Name and Score: ', artistTemplate, ', ', curScore)
            
            if curScore < lowestScore:
                bestArtist = artistTemplate
                lowestScore = curScore
                
        print('Best Template Match: ', bestArtist)        
        singerPairings[artistQuery] = (bestArtist, lowestScore)
            
    return singerPairings

In [45]:
# singerPairingsFixedW = calculateSingerFixedW(db_JingleBells, db_Templates)

In [46]:
# accuracyScoreFixedW = accuracyTest(singerPairingsFixedW)

In [47]:
# accuracyScoreFixedW

## Visualization testing

In [48]:
# for name, audio in db_Templates.items():
#     print(name)
# #     visualizeSTFT(np.abs(calcSTFT(audio)))
#     visualizeTemplate(db_Templates[name][:100,:])


In [49]:
# for name, audio in db_Scales.items():
#     print(name)
#     visualizeSTFT(np.abs(calcSTFT(audio))[:50,:])
# #     visualizeTemplate(db_Templates[name])


In [50]:
# for queryName, scaleData in singerPairings.items():
#     print('query name', queryName, 'scale name', scaleData)

In [51]:
# garyStftScaleMag = np.abs(calcSTFT(db_Scales['Gary']))
# visualizeSTFT(garyStftScaleMag[:100,:])

In [52]:
# visualizeTemplate(db_Templates['Gary'][:100,:])

In [53]:
# np.argmax(db_Templates['Gary'][:100,:], axis=0)*22050/2048

In [54]:
# garyStftJingleMag = np.abs(calcSTFT(db_JingleBells['Gary']))
# visualizeSTFT(garyStftJingleMag[:100,:])

In [55]:
# gabeSTFT= np.abs(calcSTFT(db_Scales['Gabe']))[:40,:]
# visualizeSTFT(gabeSTFT)

In [56]:
# gabeSTFT.shape

In [57]:
# idxGabeFreqs = np.argmax(gabeSTFT, axis = 1)*22050/2048

In [58]:
# print(idxGabeFreqs)